In [1]:
import django
import json,os,sys
sys.path.append("../../")
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "dva.settings")
django.setup()

In [2]:
from dvaapp import models

In [3]:
data = json.load(file('coco_sample_metadata.json'))

In [4]:
video = models.Video.objects.get(pk=1)
video.name ='mscoco_sample_500'
video.save()
models.Annotation.objects.all().filter(video=video).delete()
for frame in models.Frame.objects.all().filter(video=video):
    frame_id = str(int(frame.name.split('_')[-1].split('.')[0]))
    annotation = models.Annotation()
    annotation.video = video
    annotation.frame = frame
    annotation.full_frame = True
    annotation.metadata_text = json.dumps(data[frame_id]['image'])
    annotation.name = 'coco_image_metadata'
    annotation.label = 'image_metadata'
    annotation.save()

In [5]:
for frame in models.Frame.objects.all().filter(video=video):
    frame_id = str(int(frame.name.split('_')[-1].split('.')[0]))
    for a in data[frame_id][u'annotations']:                                                            
        annotation = models.Annotation()
        annotation.video = video
        annotation.frame = frame
        annotation.metadata_text = json.dumps(a)
        annotation.full_frame = False        
        annotation.x = a['bbox'][0]
        annotation.y = a['bbox'][1]
        annotation.w = a['bbox'][2]
        annotation.h = a['bbox'][3]
        label,_ = models.VLabel.objects.get_or_create(label_name='{}/{}'.format(a[u'category'][u'supercategory'],a[u'category'][u'name']))
        annotation.label = label.label_name
        annotation.label_parent = label
        annotation.name = 'coco_instance_annotation'
        annotation.save()